# Geographic data: Translating shapefiles to GEO DataFrames

The Census Bureau provides geographic information in *shapefile* format. In this notebook, we'll turn the raw shapefiles for NY State census tracts and "places" we downloaded earlier into clean, ready-to-map GEO DataFrames. We'll incorporate population density into our NY State census tract GEO DataFrame, save a separate GEO DataFrame for NYC, then export our GEO DataFrames for use in our next notebook on mapping.

These shapefiles are *huge* and there are a lot of them, so I don't put them in Github. Therefore, this notebook won't run in binder. but the code is here for viewing. I've commented out the code to save GEO DataFrames as Parquet files (because Arrow format for shape data is not final, so may cause problems. I plan to come back to this...)

#### *Data sources*

Files downloaded from Census Bureau FTP server via FTP client, file locations provided below.

* 2020 Census Redistricting Data (P.L. 94-171) Shapefiles, census tract level, NY State
  - file location: ftp2.census.gov/geo/tiger/TIGER2020PL/LAYER/TRACT/2020
  - file names beginning with "tl_2020_36" (NY State FIPS code is 36)

* 2020 Census Redistricting Data (P.L. 94-171) Shapefiles, places level, NY State
  - file location: ftp2.census.gov/geo/tiger/TIGER2020PL/LAYER/PLACE/2020
  - file name: tl_2020_36_place20.zip  (36 for NY State)

#### *Up close: data cleaning and mapping*

The notebooks below provide a detailed look at the other stages of cleaning and mapping the data for this project. The final steps of creating a SQL database of indoor farms in New York City and then mapping them will follow shortly.

* [Introduction: The State of Indoor Farming in the US] (00_IndoorAgriculture_start.ipynb) 
* [Data Cleanup: US Census population data](01_DataCleanup_Population.ipynb)
* [Data Cleanup: GEOID tables](02_DataCleanup_GEOIDs.ipynb)
<!-- * [Data Cleanup: Geographic data](03_DataCleanup_GIS.ipynb) -->
* [Mapping NY State: Population Density](04_Mapping_Population.ipynb)
* SQL Database Creation: Indoor Farms - *coming soon...*
* Mapping NY State: Indoor Farms - *coming soon...*

#### *Some helper code for managing file paths*
We'll use the code below throughout this project to make it easier to refer to the folders where our various data files are stored.

In [1]:
# os and patlib modules used to make it easier to refer to project folders 

import os, pathlib
base_dir = pathlib.Path(os.getcwd()).parent
data_archive_dir = os.path.join(base_dir, "data_archive")
clean_data_dir = os.path.join(data_archive_dir, "clean")
data_dir = os.path.join(base_dir, "data")
shapes_dir = os.path.join(data_dir,"shapes")
json_dir = os.path.join(data_dir,"geojson")
util_dir = os.path.join(data_dir,"util")

## Census tract shapefiles to GEO DataFrame

 Let's load our NY State census tract shapefiles as a GEO DataFrame and call it *geodf_tract_ny*.

In [2]:
import geopandas as gpd

In [3]:
big_tract_ny_shapefile = os.path.join(shapes_dir,"tl_2020_36_tract20.zip") # provide the full path to our shapefiles

# uses the geopandas function read_file to grab our file

# geodf_tract_ny = gpd.read_file(big_tract_ny_shapefile)[['STATEFP20', 'COUNTYFP20', 'TRACTCE20', 'GEOID20', 'ALAND20',  'AWATER20', 'INTPTLAT20', 'INTPTLON20', 'geometry']] 
geodf_tract_ny = gpd.read_file(big_tract_ny_shapefile)[['STATEFP20', 'COUNTYFP20', 'TRACTCE20', 'GEOID20', 'ALAND20',  'AWATER20', 'geometry']] 
# geodf_tract_ny = gpd.read_file(big_tract_ny_shapefile)
# import pyarrow as pa
# import warnings; 
# warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
# geodf_tract_ny.to_parquet(small_clean_shapefile, index=False, compression='BROTLI')
#  #= pa.Table.from_pandas(ny_shapes_df, preserve_index=False)

In [4]:
# rename the columns
# geodf_tract_ny = geodf_tract_ny.rename(columns={'STATEFP20': 'State FIPS', 'COUNTYFP20': 'County FIPS', 'TRACTCE20': 'Census Tract Code', 'GEOID20': 'GEOID Census Tract', 'ALAND20': 'Land Area', 'AWATER20': 'Water Area', 'INTPTLAT20': 'Latitude of Internal Point', 'INTPTLON20': 'Longitude of Internal Point'})
geodf_tract_ny = geodf_tract_ny.rename(columns={'STATEFP20': 'State FIPS', 'COUNTYFP20': 'County FIPS', 'TRACTCE20': 'Census Tract Code', 'GEOID20': 'GEOID Census Tract', 'ALAND20': 'Land Area', 'AWATER20': 'Water Area'})

# set the data types of each column as we want them to be
geodf_tract_ny = geodf_tract_ny.astype({'State FIPS': 'int', 'County FIPS':'int', 'Census Tract Code':'int', 'GEOID Census Tract': 'int', 'Land Area': 'int', 'Water Area': 'int'})

# NY State had 5411 census tracts total for the 2020 census (https://www.census.gov/geographies/reference-files/time-series/geo/tallies.html#tract_bg_block)
# so we want to see .shape to return 5411 rows below
geodf_tract_ny.shape 

(5411, 7)

In [5]:
geodf_tract_ny.head(3)

,State FIPS,County FIPS,Census Tract Code,GEOID Census Tract,Land Area,Water Area,geometry
0,36,47,700,36047000700,176774,0,"POLYGON ((-74.00154 40.69279, -74.00132 40.693..."
1,36,47,900,36047000900,163469,0,"POLYGON ((-73.99405 40.69090, -73.99374 40.691..."
2,36,47,1100,36047001100,168507,0,"POLYGON ((-73.99073 40.69305, -73.99045 40.693..."


As usual, we'll conduct our checks for correct data types and missing values.

In [6]:
geodf_tract_ny.info()
geodf_tract_ny['geometry'].dtype

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 5411 entries, 0 to 5410
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   State FIPS          5411 non-null   int64   
 1   County FIPS         5411 non-null   int64   
 2   Census Tract Code   5411 non-null   int64   
 3   GEOID Census Tract  5411 non-null   int64   
 4   Land Area           5411 non-null   int64   
 5   Water Area          5411 non-null   int64   
 6   geometry            5411 non-null   geometry
dtypes: geometry(1), int64(6)
memory usage: 296.0 KB


In [7]:
# double check for null values
geodf_tract_ny[geodf_tract_ny.isna().any(axis=1)]

,State FIPS,County FIPS,Census Tract Code,GEOID Census Tract,Land Area,Water Area,geometry


No null values! We're making progress - our big NY Census Tract shapefile is now a nice *clean* GEO DataFrame.

### Merging population data and geographic data

We're now ready to use the population data we so carefully cleaned in our first notebook! Let's beging by loading as a pandas DataFrame the clean population data we saved earlier to parquet format. 

In [8]:
import pyarrow.parquet as pq

clean_census_pop_file_parquet = os.path.join(clean_data_dir, 'census_pop.parquet')
df_census_pop = pq.read_table(clean_census_pop_file_parquet, memory_map=True).to_pandas()
df_census_pop.head(3)

,GEOID Census Tract Full,Census Tract Name,Population,GEOID Census Tract
0,1400000US01001020100,"Census Tract 201, Autauga County, Alabama",1775,1001020100
1,1400000US01001020200,"Census Tract 202, Autauga County, Alabama",2055,1001020200
2,1400000US01001020300,"Census Tract 203, Autauga County, Alabama",3216,1001020300


Now we can go ahead and merge *df_census_pop* into *geodf_tract_ny* on column "GEOID Census Tract". (We'll be calculating population density shortly, but first we need to check a few things...)

In [9]:
geodf_tract_ny = geodf_tract_ny.merge(df_census_pop, on=['GEOID Census Tract'])
geodf_tract_ny = geodf_tract_ny[['State FIPS', 'County FIPS', 'Census Tract Name', 'GEOID Census Tract', 'Population', 'Land Area', 'Water Area', 'geometry']]
geodf_tract_ny.head(3)

,State FIPS,County FIPS,Census Tract Name,GEOID Census Tract,Population,Land Area,Water Area,geometry
0,36,47,"Census Tract 7, Kings County, New York",36047000700,4415,176774,0,"POLYGON ((-74.00154 40.69279, -74.00132 40.693..."
1,36,47,"Census Tract 9, Kings County, New York",36047000900,5167,163469,0,"POLYGON ((-73.99405 40.69090, -73.99374 40.691..."
2,36,47,"Census Tract 11, Kings County, New York",36047001100,1578,168507,0,"POLYGON ((-73.99073 40.69305, -73.99045 40.693..."


### Excluding census tracts that are water only

For reasons we plan to research further at some point, the US Census includes some census tracts that consist entirely of water and no land. It stands to reason that these would not have any permanent residents and would, therefore, have a recorded population of 0. If this is the case, we'll want to exclude them from our population density choropleth as they will create unecessary noise.

But, to be safe, let's test our assumption. First, let's see how many census tracts in NY state consist entirely of water.

In [10]:
geodf_tract_ny[geodf_tract_ny['Land Area']==0].count()

State FIPS            18
County FIPS           18
Census Tract Name     18
GEOID Census Tract    18
Population            18
Land Area             18
Water Area            18
geometry              18
dtype: int64

Apparently, there are a total of 18 census tracts in NY state than consist of water only! Now, let's see if we're correct that all of these have a population of 0.

In [11]:
geodf_tract_ny[(geodf_tract_ny['Land Area'] == 0) & (geodf_tract_ny['Population'] == 0)]

,State FIPS,County FIPS,Census Tract Name,GEOID Census Tract,Population,Land Area,Water Area,geometry
127,36,55,"Census Tract 9900, Monroe County, New York",36055990000,0,0,1799140350,"POLYGON ((-77.99478 43.37506, -77.99402 43.500..."
133,36,81,"Census Tract 9901, Queens County, New York",36081990100,0,0,130800077,"POLYGON ((-74.03813 40.53829, -74.03749 40.542..."
732,36,45,"Census Tract 9900.01, Jefferson County, New York",36045990001,0,0,1366058435,"POLYGON ((-76.70129 43.75006, -76.63362 43.834..."
788,36,75,"Census Tract 9900, Oswego County, New York",36075990000,0,0,724225049,"POLYGON ((-76.64471 43.70347, -76.62468 43.702..."
846,36,47,"Census Tract 9901, Kings County, New York",36047990100,0,0,17793788,"POLYGON ((-74.04374 40.60669, -74.04348 40.606..."
1242,36,59,"Census Tract 9901, Nassau County, New York",36059990100,0,0,13640682,"POLYGON ((-73.78019 40.82640, -73.77498 40.830..."
1243,36,59,"Census Tract 9903.02, Nassau County, New York",36059990302,0,0,37600401,"POLYGON ((-73.48439 40.55268, -73.48479 40.589..."
1319,36,59,"Census Tract 9903.01, Nassau County, New York",36059990301,0,0,67474483,"POLYGON ((-73.61309 40.95082, -73.61300 40.950..."
1393,36,73,"Census Tract 9900, Orleans County, New York",36073990000,0,0,1099598019,"POLYGON ((-78.46871 43.63343, -78.38882 43.633..."
1481,36,59,"Census Tract 9902, Nassau County, New York",36059990200,0,0,27086403,"POLYGON ((-73.69445 40.90305, -73.62815 40.941..."


Confirmed - there are, in fact, 0 residents in any NY state census tract thats consist of water only. Now we can go ahead and exclude these census tracts from our Geo DataFrame. 

In [12]:
geodf_tract_ny = geodf_tract_ny[(geodf_tract_ny['Land Area'] != 0)]

In [13]:
# add a column to calculate population density
# land area measured in whole sq meters
# sq miles = sq meters/2,589,988 
# population density in sq miles  = population/ (land area sq meters / 2,589,988)
geodf_tract_ny['Population Density'] = (geodf_tract_ny['Population']/(geodf_tract_ny['Land Area']/2589988)).round(0)
geodf_tract_ny = geodf_tract_ny[['State FIPS', 'County FIPS', 'Census Tract Name', 'GEOID Census Tract', 'Population', 'Land Area', 'Water Area', 'Population Density', 'geometry']]
geodf_tract_ny.head(3)

,State FIPS,County FIPS,Census Tract Name,GEOID Census Tract,Population,Land Area,Water Area,Population Density,geometry
0,36,47,"Census Tract 7, Kings County, New York",36047000700,4415,176774,0,64686.0,"POLYGON ((-74.00154 40.69279, -74.00132 40.693..."
1,36,47,"Census Tract 9, Kings County, New York",36047000900,5167,163469,0,81865.0,"POLYGON ((-73.99405 40.69090, -73.99374 40.691..."
2,36,47,"Census Tract 11, Kings County, New York",36047001100,1578,168507,0,24254.0,"POLYGON ((-73.99073 40.69305, -73.99045 40.693..."


Our population density calculation will have produced a NaN value in rows where "Population" == 0. So, let's locate these and replace each "NaN" in the "Population Density" column with a "0".

In [14]:
# tracts with population == 0 because these will result in NaN when we calculate Population Density
# so let's find these and replace the NaNs with 0s
geodf_tract_ny[geodf_tract_ny['Population'] == 0]
geodf_tract_ny['Population Density'] = geodf_tract_ny['Population Density'].fillna(0)
geodf_tract_ny['Population Density'] = geodf_tract_ny['Population Density'].astype(int) # change dtype to integer

Now let's take it a step further and also make our clean DataFrame *smaller* by saving it to Parquet format.

### Saving our clean GEO DataFrame for later

In [15]:
import pyarrow as pa
import warnings; 
warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
clean_geodf_tract_ny_file_parquet = os.path.join(clean_data_dir, "tl_2020_36_tract20.parquet")
geodf_tract_ny.to_parquet(clean_geodf_tract_ny_file_parquet, index=False, compression='BROTLI')
#= pa.Table.from_pandas(ny_shapes_df, preserve_index=False)

In [16]:
# we'll also save a CSV version 
clean_geodf_tract_ny_file_csv  = os.path.join(clean_data_dir, "tl_2020_36_tract20.csv")
geodf_tract_ny.to_csv(clean_geodf_tract_ny_file_csv, index=False)
#= pa.Table.from_pandas(ny_shapes_df, preserve_index=False)

## NY State "Place" shapefiles to GEO DataFrame

Let's load the NY State "Place" shapefiles as a GEO DataFrame and call it *geodf_place_ny*.

In [17]:
import geopandas as gpd 

# make GEO DataFrame from the NY State "Place" shapefiles
shapefile_place_ny = os.path.join(shapes_dir,"tl_2020_36_place20.zip")
geodf_place_ny = gpd.read_file(shapefile_place_ny, encoding_errors='ignore')

# select only the columns we want 
geodf_place_ny = geodf_place_ny[['STATEFP20', 'PLACEFP20', 'GEOID20', 'NAME20', 'ALAND20', 'AWATER20', 'geometry']]

# rename the columns
geodf_place_ny.rename(columns={'STATEFP20': 'State FIPS', 'PLACEFP20': 'Place FIPS', 'GEOID20': 'GEOID', 'NAME20': 'Name', 'ALAND20': 'Land Area', 'AWATER20': 'Water Area'}, inplace=True)

geodf_place_ny.head(3)
geodf_place_ny

,State FIPS,Place FIPS,GEOID,Name,Land Area,Water Area,geometry
0,36,01517,3601517,Altamont,3279144,3689,"MULTIPOLYGON (((-74.02271 42.70420, -74.02231 ..."
1,36,56979,3656979,Peekskill,11251161,3177119,"POLYGON ((-73.95544 41.27786, -73.95204 41.280..."
2,36,18388,3618388,Cortland,10085376,51890,"POLYGON ((-76.20049 42.61248, -76.19596 42.612..."
3,36,35276,3635276,Homer,4986794,32383,"POLYGON ((-76.20248 42.63292, -76.20161 42.633..."
4,36,42147,3642147,Lewiston,2840234,360474,"POLYGON ((-79.05292 43.17400, -79.05287 43.177..."
...,...,...,...,...,...,...,...
1288,36,83426,3683426,Yaphank,35336639,318022,"POLYGON ((-72.96103 40.85172, -72.96021 40.851..."
1289,36,84011,3684011,York,7133085,0,"POLYGON ((-77.90695 42.87215, -77.90689 42.873..."
1290,36,84044,3684044,Yorkshire,4778096,26720,"POLYGON ((-78.49301 42.51619, -78.49283 42.519..."
1291,36,84088,3684088,Yorktown Heights,2379367,8652,"POLYGON ((-73.78428 41.27440, -73.78427 41.274..."


Are there any "places" in NY state that consist of water only and no land?

In [18]:
geodf_place_ny[geodf_place_ny['Land Area'] == 0]

,State FIPS,Place FIPS,GEOID,Name,Land Area,Water Area,geometry


And, of course, we'll conduct our checks for missing values and correct datatypes.

In [19]:
geodf_place_ny.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1293 entries, 0 to 1292
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   State FIPS  1293 non-null   object  
 1   Place FIPS  1293 non-null   object  
 2   GEOID       1293 non-null   object  
 3   Name        1293 non-null   object  
 4   Land Area   1293 non-null   int64   
 5   Water Area  1293 non-null   int64   
 6   geometry    1293 non-null   geometry
dtypes: geometry(1), int64(2), object(4)
memory usage: 70.8+ KB


The data type for State FIPS, Place FIPS, and GEOID should really be integer. Let's fix that now.

In [20]:
# Set the data type of each column are as we want it to be
geodf_place_ny['State FIPS'] = geodf_place_ny['State FIPS'].astype(int)
geodf_place_ny['Place FIPS'] = geodf_place_ny['Place FIPS'].astype(int)
geodf_place_ny['GEOID'] = geodf_place_ny['GEOID'].astype(int)
geodf_place_ny.dtypes

State FIPS       int64
Place FIPS       int64
GEOID            int64
Name            object
Land Area        int64
Water Area       int64
geometry      geometry
dtype: object

One final check for missing values.

In [21]:
# another check for null values
geodf_place_ny[geodf_place_ny.isna().any(axis=1)]

,State FIPS,Place FIPS,GEOID,Name,Land Area,Water Area,geometry


All clear! With null values rule out, we're almost done with our NY State place GEO DataFrame. There's just one last thing bothering us - a naming inconsistency between the NY place FIPS code list and NY place shapefiles we downloaded. This is confusing!

### Ensuring consistent naming

We can see an example below when we look at place FIPS 51000. It's referred to as "New York City" in the the NY Place FIPS code list and simply "New York" in the corresponding shapefiles. 

In [22]:
import pyarrow.parquet as pq

clean_df_place_file = os.path.join(clean_data_dir, 'place.parquet')
df_place = pq.read_table(clean_df_place_file, memory_map=True).to_pandas()
df_place[(df_place['Place FIPS'] == 51000) & (df_place['State FIPS'] == 36)]

,State Name,State,State FIPS,Place,Place FIPS,County
24316,New York,NY,36,New York City,51000,"Bronx County, Kings County, New York County, Q..."


In [23]:
geodf_place_ny[geodf_place_ny['Place FIPS'] == 51000]

,State FIPS,Place FIPS,GEOID,Name,Land Area,Water Area,geometry
398,36,51000,3651000,New York,778165283,445421632,"MULTIPOLYGON (((-74.04075 40.70017, -74.04073 ..."


We prefer the full name, "New York City". So, we'll resolve the inconsistency by merging the "Place" column from *df_place* into *geodf_place_ny* and keeping only the columns we want. 

In [24]:
geodf_place_ny = geodf_place_ny.merge(df_place, on =['State FIPS', 'Place FIPS'])
geodf_place_ny = geodf_place_ny[['State Name', 'State', 'State FIPS', 'Place', 'Place FIPS', 'County', 'Land Area', 'Water Area', 'geometry']] 
geodf_place_ny[geodf_place_ny['Place FIPS'] == 51000]

,State Name,State,State FIPS,Place,Place FIPS,County,Land Area,Water Area,geometry
440,New York,NY,36,New York City,51000,"Bronx County, Kings County, New York County, Q...",778165283,445421632,"MULTIPOLYGON (((-74.04075 40.70017, -74.04073 ..."


Good! We can now save this clean, consistent version as both a CSV and a Parquet file for use in our next notebook.

In [25]:
# save geodf_place_ny as Parquet
clean_geodf_place_ny_file_parquet = os.path.join(clean_data_dir, 'geodf_place_ny.parquet')
geodf_place_ny.to_parquet(clean_geodf_place_ny_file_parquet, index=False, compression='BROTLI')

In [26]:
# this is a back up CSV version 
clean_geodf_place_ny_file_csv  = os.path.join(clean_data_dir, "geodf_place_ny.csv")
geodf_place_ny.to_csv(clean_geodf_place_ny_file_csv, index=False)
#= pa.Table.from_pandas(ny_shapes_df, preserve_index=False)

## New York City "place" GEO DataFrame

Lastly, we'll save a separate GEO DataFrame of just the geographic information for New York City. We'll call it *geodfy_place_nyc* and also save it as a CSV and Parquet file. 

In [27]:
# make a new GEO DF of just the Place New York City
geodf_place_nyc = geodf_place_ny[geodf_place_ny['Place FIPS'] == 51000]
geodf_place_nyc 

,State Name,State,State FIPS,Place,Place FIPS,County,Land Area,Water Area,geometry
440,New York,NY,36,New York City,51000,"Bronx County, Kings County, New York County, Q...",778165283,445421632,"MULTIPOLYGON (((-74.04075 40.70017, -74.04073 ..."


In [28]:
# save as Parquet
clean_geodf_place_nyc_file_parquet = os.path.join(clean_data_dir, 'geodf_place_nyc.parquet')
geodf_place_nyc.to_parquet(clean_geodf_place_nyc_file_parquet, index=False, compression='BROTLI')

In [29]:
# also save a CSV version 
clean_geodf_place_nyc_file_csv = os.path.join(clean_data_dir, "geodf_place_nyc.csv")
geodf_place_nyc.to_csv(clean_geodf_place_nyc_file_csv, index=False)
#= pa.Table.from_pandas(ny_shapes_df, preserve_index=False)